In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/DataEngineeringQ2.json')
df.head()

,_id,appointmentId,patientDetails,phoneNumber,consultationData
0,T6hf3rb5,40d2-9c9f,"{'_id': 'T6hb630b3', 'firstName': 'Css', 'last...",96686896670,"{'adviceTemplates': [], 'advices': [], 'attach..."
1,T65g3rb5,40dbtc9f,"{'_id': 'T6h33b300', 'firstName': 'Lokesh', 'l...",9496368916,"{'adviceTemplates': [], 'advices': [], 'attach..."
2,T7g6Srb5,g3Wt5c9f,"{'_id': 'TjhB4373', 'phrId': '63b5hvy614d5', '...",7787204833,"{'adviceTemplates': [], 'advices': [], 'attach..."
3,94bg8W8d,fb6-a535,"{'_id': '6df4R5b', 'phrId': '644nig7y', 'first...",9376756879,"{'adviceTemplates': [], 'advices': [], 'attach..."
4,T6h8H56c,67h7KL9,"{'_id': 'lK9hy06a', 'firstName': 'Raghu Viju',...",5267384241,"{'adviceTemplates': [], 'advices': [], 'attach..."


In [ ]:
import pandas as pd


df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/DataEngineeringQ2.json')

df['firstName'] = df['patientDetails'].apply(lambda x: x.get('firstName', ''))
df['lastName'] = df['patientDetails'].apply(lambda x: x.get('lastName', ''))
df['birthDate'] = df['patientDetails'].apply(lambda x: x.get('birthDate', ''))


def calculate_missing_percentage(series):
    total = len(series)
    missing = series.apply(lambda x: x in [None, '', ' ']).sum()
    return round((missing / total) * 100, 2)


missing_firstName = calculate_missing_percentage(df['firstName'])
missing_lastName = calculate_missing_percentage(df['lastName'])
missing_birthDate = calculate_missing_percentage(df['birthDate'])


print(f"Missing percentage for firstName: {missing_firstName}%")
print(f"Missing percentage for lastName: {missing_lastName}%")
print(f"Missing percentage for birthDate: {missing_birthDate}%")


Missing percentage for firstName: 0.0%
Missing percentage for lastName: 70.97%
Missing percentage for birthDate: 32.26%


In [ ]:
import pandas as pd


df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/DataEngineeringQ2.json')

df['gender'] = df['patientDetails'].apply(lambda x: x.get('gender', ''))


mode_gender = df['gender'][df['gender'] != ''].mode()[0]
df['gender'] = df['gender'].apply(lambda x: mode_gender if x in [None, '', ' '] else x)


total = len(df)
female_count = df['gender'].apply(lambda x: x.lower() == 'f').sum()
female_percentage = round((female_count / total) * 100, 2)

print(female_percentage)


32.26


In [ ]:
import pandas as pd
from datetime import datetime

def calculate_age(row):
    patient_details = row['patientDetails']
    if 'birthDate' in patient_details:
        birth_date = patient_details['birthDate']
        if birth_date:
            birth_date = datetime.strptime(birth_date, '%Y-%m-%dT%H:%M:%S.%fZ')  # Adjust format if needed
            today = datetime.today()
            age = today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
            return age
    return None


def categorize_age(age):
    if age is None:
        return None
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'


df['age'] = df.apply(calculate_age, axis=1)
df['ageGroup'] = df['age'].apply(categorize_age)


count_adults = df[df['ageGroup'] == 'Adult'].shape[0]


print(count_adults)


21


In [ ]:
import pandas as pd
import json




def count_medicines(row):
    consultation_data = row['consultationData']
    if 'medicines' in consultation_data:
        return len(consultation_data['medicines'])
    else:
        return 0


df['numMedicines'] = df.apply(count_medicines, axis=1)

average_medicines = df['numMedicines'].mean()


average_medicines_rounded = round(average_medicines, 2)


print(average_medicines_rounded)


2.13


In [ ]:
import pandas as pd
import json
from collections import Counter


def extract_and_count_medicine_names(row):
    consultation_data = row['consultationData']
    if 'medicines' in consultation_data:
        return [medicine['medicineName'] for medicine in consultation_data['medicines']]
    else:
        return []


df['medicineNames'] = df.apply(extract_and_count_medicine_names, axis=1)


medicine_names_list = [medicine for sublist in df['medicineNames'] for medicine in sublist]

medicine_name_counts = Counter(medicine_names_list)


third_most_common_medicine = medicine_name_counts.most_common(3)[-1][0]


print(third_most_common_medicine)


C


In [ ]:
import pandas as pd
import json


def count_active_inactive_medicines(row):
    consultation_data = row['consultationData']
    if 'medicines' in consultation_data:
        active_count = sum(1 for medicine in consultation_data['medicines'] if medicine['isActive'])
        inactive_count = sum(1 for medicine in consultation_data['medicines'] if not medicine['isActive'])
    else:
        active_count = 0
        inactive_count = 0
    return active_count, inactive_count


df[['activeMedicines', 'inactiveMedicines']] = df.apply(count_active_inactive_medicines, axis=1, result_type='expand')

total_active = df['activeMedicines'].sum()
total_inactive = df['inactiveMedicines'].sum()


percentage_active = (total_active / (total_active + total_inactive)) * 100
percentage_inactive = (total_inactive / (total_active + total_inactive)) * 100


print(f"{percentage_active:.2f}, {percentage_inactive:.2f}")


69.70, 30.30


In [ ]:
import pandas as pd


def is_valid_indian_phone_number(phone_number):

    phone_number = phone_number.replace(' ', '').replace('-', '')


    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]  # Remove '+91 '
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]  # Remove '91'
    else:
        return False


    if len(phone_number) != 10 or not phone_number.isdigit():
        return False

    if 6000000000 <= int(phone_number) <= 9999999999:
        return True
    else:
        return False


df['isValidMobile'] = df['phoneNumber'].apply(is_valid_indian_phone_number)


valid_phone_count = df['isValidMobile'].sum()


print(valid_phone_count)


6


In [ ]:
import pandas as pd
from scipy.stats import pearsonr


data = {
    'consultationData': [
        {'medicines': [{'medicineName': 'A'}, {'medicineName': 'B'}]},
        {'medicines': [{'medicineName': 'C'}]},
        {'medicines': []}
    ],
    'birthDate': ['1990-01-01', '1985-05-15', '1970-12-25']
}
df = pd.DataFrame(data)


def count_prescribed_medicines(consultation_data):
    if 'medicines' in consultation_data:
        return len(consultation_data['medicines'])
    else:
        return 0


df['prescribedMedicines'] = df['consultationData'].apply(count_prescribed_medicines)

def calculate_age(birth_date):

    return 2024 - int(birth_date[:4])


df['age'] = df['birthDate'].apply(calculate_age)


pearson_corr, _ = pearsonr(df['prescribedMedicines'], df['age'])


pearson_corr_rounded = round(pearson_corr, 2)


print(pearson_corr_rounded)


-0.96
